# Goal of the Study

Study and analyze the data on books nad only bookstore to make a proposition for a new Product

## Description of the data

**books**

Contains data on books:

- `book_id`
- `author_id`
- `title`
- `num_pages` — number of pages
- `publication_date`
- `publisher_id`

**authors**

Contains data on authors:

- `author_id`
- `author`

**publishers**

Contains data on publishers:

- `publisher_id`
- `publisher`

**ratings**

Contains data on user ratings:

- `rating_id`
- `book_id`
- `username` — the name of the user who rated the book
- `rating`

**reviews**

Contains data on customer reviews:

- `review_id`
- `book_id`
- `username` — the name of the user who reviewed the book
- `text` — the text of the review

### Read the Data

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
import pandas as pd
from sqlalchemy import create_engine

In [3]:
# creating connection with the database
db_config = {'user': 'praktikum_student',         # user name
             'pwd': 'password', # password
             'host': 'host',
             'port': 'port',              # connection port
             'db': 'data-analyst-final-project-db'}          # the name of the data base

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [4]:
#function to return query
def save_set(data):
    query = f''' SELECT *
            FROM {data}
        '''
    return pd.io.sql.read_sql(query, con = engine)

In [5]:
#loop to save the sets in variables
for i in ['books', 'authors', 'publishers', 'ratings', 'reviews']:
    globals()[i] = save_set(i)
    print(f'{i}:') 
    globals()[i].head()

books:


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


authors:


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


publishers:


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


ratings:


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


reviews:


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Task analysis

### 1.Find the number of books released after January 1, 2000.

In [6]:
query_1 =  '''
SELECT
    COUNT(DISTINCT book_id)
FROM books
WHERE (publication_date :: date) > '2000-01-01'

'''

books_released = pd.io.sql.read_sql(query_1, con = engine)
books_released

,count
0,819


- We have 819 books in the database that were released in the 21. Century starting (01.01.2000)

### 2.Find the number of user reviews and the average rating for each book.

In [7]:
query_2_1 =  '''
SELECT
    books.book_id,
    COUNT(DISTINCT reviews.review_id) AS review_cnt,
    AVG(ratings.rating) AS avg_rating
FROM books
LEFT JOIN reviews ON (books.book_id) = (reviews.book_id)
LEFT JOIN ratings ON (books.book_id) = (ratings.book_id)
GROUP BY
    books.book_id
ORDER BY
    avg_rating DESC
'''

reviews_and_rating_1 = pd.io.sql.read_sql(query_2_1, con = engine)
reviews_and_rating_1

,book_id,review_cnt,avg_rating
0,86,2,5.00
1,901,2,5.00
2,390,2,5.00
3,972,2,5.00
4,136,2,5.00
...,...,...,...
995,915,3,2.25
996,316,2,2.00
997,202,3,2.00
998,371,2,2.00


In [8]:
query_2_2 =  '''
SELECT
    books.book_id,
    COUNT(DISTINCT reviews.review_id) AS review_cnt,
    AVG(ratings.rating) AS avg_rating
FROM books
LEFT JOIN reviews ON (books.book_id) = (reviews.book_id)
LEFT JOIN ratings ON (books.book_id) = (ratings.book_id)
GROUP BY
    books.book_id
ORDER BY
    review_cnt DESC
'''

reviews_and_rating_2 = pd.io.sql.read_sql(query_2_2, con = engine)
reviews_and_rating_2

,book_id,review_cnt,avg_rating
0,948,7,3.662500
1,963,6,3.977273
2,734,6,4.206897
3,302,6,4.414634
4,695,6,4.081081
...,...,...,...
995,83,0,3.666667
996,808,0,3.000000
997,672,0,5.000000
998,221,0,4.000000


 - We Have 1000 Books  in our data base.
 - The average ratings range from 5 to 1.5 
 - There are books that were not reviewed from Users and the most reviewed book has 7 reviews
 
 
 ### 3.Identify the publisher that has released the greatest number of books with more than 50 pages 

In [9]:
query_3 =  '''
    SELECT
        publishers.publisher AS publisher,
        COUNT(DISTINCT books.book_id) AS books_cnt
    FROM books
    LEFT JOIN publishers ON (books.publisher_id) = (publishers.publisher_id)
    WHERE books.num_pages >= 50 
    GROUP BY
        publisher
    ORDER BY
        books_cnt DESC
    LIMIT 5
'''

best_publisher = pd.io.sql.read_sql(query_3, con = engine)
best_publisher

,publisher,books_cnt
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19


- We see that the Publisher who published the greatest number of books is `Penguin Books` with 42 books which have more 50 Pages
- Next comes `Vintage` then `Grand Central Publishing` 


### 4.Identify the author with the highest average book rating (look only at books with at least 50 ratings).

In [10]:
query_4_1 =  '''
    SELECT
        authors.author,
        Sub_rating.AVG_rating
    FROM (
        SELECT
            books.author_id AS author_id,
            books.book_id AS book_id,
            AVG(ratings.rating) AS AVG_rating
        FROM books
        LEFT JOIN ratings ON (books.book_id) = (ratings.book_id)
        GROUP BY
            books.book_id
        HAVING 
            COUNT(ratings.rating) >= 50 ) AS Sub_rating
    LEFT JOIN authors on (Sub_rating.author_id) = (authors.author_id)
    ORDER BY
        AVG_rating DESC
'''

best_author_1 = pd.io.sql.read_sql(query_4_1, con = engine)
best_author_1

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.414634
1,J.R.R. Tolkien,4.391892
2,J.K. Rowling/Mary GrandPré,4.287500
3,Markus Zusak/Cao Xuân Việt Khương,4.264151
4,J.K. Rowling/Mary GrandPré,4.246575
5,Louisa May Alcott,4.192308
6,J.K. Rowling/Mary GrandPré,4.186667
7,J.R.R. Tolkien,4.125000
8,Rick Riordan,4.080645
9,William Golding,3.901408


In [11]:
query_4_2 =  '''
    SELECT
        authors.author,
        AVG(Sub_rating.book_rating) AS AVG_rating
    FROM (
        SELECT
            books.author_id AS author_id,
            books.book_id AS book_id,
            AVG(ratings.rating) AS book_rating
        FROM books
        LEFT JOIN ratings ON (books.book_id) = (ratings.book_id)
        GROUP BY
            books.book_id
        HAVING 
            COUNT(ratings.rating) >= 50 ) AS Sub_rating
    LEFT JOIN authors on (Sub_rating.author_id) = (authors.author_id)
    GROUP BY
        Sub_rating.author_id,
        authors.author
    ORDER BY
        AVG_rating DESC
'''

best_author_2 = pd.io.sql.read_sql(query_4_2, con = engine)
best_author_2

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


- We can see that `J.K. Rowling/Mary GrandPré` have the highest average ratings for their books with an average of 4.28 followed closely by `Markus Zusak/Cao Xuân Việt Khương` then `J.R.R. Tolkien` 
- Despite the small difference in average ratings for their books but `J.K. Rowling/Mary GrandPré`are dominating the the rating standing with 3 books in the top 5 highest rated books

### 5.Find the average number of text reviews among users who rated more than 50 books. 

In [12]:
query_5 =  '''
   
SELECT
    AVG(Sub.review_count) AS avg_reviews
FROM (
    SELECT 
        reviews.username AS User,
        COUNT(DISTINCT reviews.review_id) AS review_count,
        COUNT(DISTINCT ratings.rating_id) AS rating_count
    FROM reviews
    LEFT JOIN ratings ON reviews.username = ratings.username
    GROUP BY
        reviews.username ) AS Sub
WHERE Sub.rating_count > 50


'''

avg_reviews = pd.io.sql.read_sql(query_5, con = engine)
avg_reviews

,avg_reviews
0,24.333333


- We can see that the average number of text reviews among users who rated more than 50 books is 24 reviews which can be interpreted that for every 2 books the user